<a href="https://colab.research.google.com/github/azarenkova-git/Fire-and-smoke-detection/blob/main/fire_and_smoke_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd "/gdrive/MyDrive/работа/fire_and_smoke_detection"

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/MyDrive/работа/fire_and_smoke_detection


In [ ]:
import os
import random
import shutil
import yaml
from ultralytics import YOLO

# Функция для преобразования bbox из COCO-формата в YOLO-формат



In [ ]:
def convert_bbox(image_size, bbox):
    width, height = image_size
    x, y, w, h = bbox

    x_center = x + w / 2.0
    y_center = y + h / 2.0

    x_center_norm = x_center / width
    y_center_norm = y_center / height
    w_norm = w / width
    h_norm = h / height

    return x_center_norm, y_center_norm, w_norm, h_norm


def process_annotations(json_path, output_dir, valid_categories):
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    images_info = {img['id']: img for img in data['images']}

    annotations_per_image = {}
    for ann in data['annotations']:
        cat_id = ann['category_id']
        if cat_id in valid_categories:
            image_id = ann['image_id']
            if image_id not in annotations_per_image:
                annotations_per_image[image_id] = []
            annotations_per_image[image_id].append(ann)

    os.makedirs(output_dir, exist_ok=True)

    for image_id, img_info in images_info.items():
        file_name = img_info['file_name']
        img_width = img_info['width']
        img_height = img_info['height']

        base_name = os.path.splitext(os.path.basename(file_name))[0]
        txt_path = os.path.join(output_dir, base_name + '.txt')

        ann_list = annotations_per_image.get(image_id, [])

        with open(txt_path, 'w') as out_file:
            for ann in ann_list:
                bbox = ann['bbox']
                x_c, y_c, w_n, h_n = convert_bbox((img_width, img_height), bbox)

                new_cat_id = valid_categories[ann['category_id']]
                line = f"{new_cat_id} {x_c:.6f} {y_c:.6f} {w_n:.6f} {h_n:.6f}\n"
                out_file.write(line)

train_json_path = os.path.join("data", "475_fire_train", "annotations", "instances_default.json")
train_output_dir = os.path.join("data", "475_fire_train", "labels")

val_json_path = os.path.join("data", "474_fire_val", "annotations", "instances_default.json")
val_output_dir = os.path.join("data", "474_fire_val", "labels")


process_annotations(train_json_path, train_output_dir, valid_categories)
process_annotations(val_json_path, val_output_dir, valid_categories)

# Перемещаю данные из train и val в предыдущие папки


In [ ]:
train_dir = os.path.join("data", "475_fire_train", "images", "train")
train_target = os.path.join("data", "475_fire_train", "images")

val_dir = os.path.join("data", "474_fire_val", "images", "val")
val_target = os.path.join("data", "474_fire_val", "images")

def move_images(src_dir, dst_dir):
    if not os.path.exists(src_dir):

        return
    if not os.path.exists(dst_dir):
        os.makedirs(dst_dir, exist_ok=True)


    for filename in os.listdir(src_dir):
        src_file = os.path.join(src_dir, filename)
        dst_file = os.path.join(dst_dir, filename)
        if os.path.isfile(src_file):
            shutil.move(src_file, dst_file)

    if not os.listdir(src_dir):
        os.rmdir(src_dir)

move_images(train_dir, train_target)
move_images(val_dir, val_target)

In [ ]:
!pip install ultralytics

# Разделение данных train на две части - train и dev, в соотношении 80/20

In [ ]:
def split_train_dev(source_images_dir, source_labels_dir, dest_train_images, dest_train_labels, dest_dev_images, dest_dev_labels, dev_ratio=0.2, seed=42):

    os.makedirs(dest_train_images, exist_ok=True)
    os.makedirs(dest_train_labels, exist_ok=True)
    os.makedirs(dest_dev_images, exist_ok=True)
    os.makedirs(dest_dev_labels, exist_ok=True)

    image_files = [f for f in os.listdir(source_images_dir) if f.lower().endswith('.jpg')]
    image_files.sort()
    random.seed(seed)
    random.shuffle(image_files)

    num_images = len(image_files)
    num_dev = int(num_images * dev_ratio)

    dev_files = image_files[:num_dev]
    train_files = image_files[num_dev:]

    for file in train_files:
        src_img_path = os.path.join(source_images_dir, file)
        dest_img_path = os.path.join(dest_train_images, file)
        shutil.copy(src_img_path, dest_img_path)

        label_file = file.rsplit('.', 1)[0] + '.txt'
        src_label_path = os.path.join(source_labels_dir, label_file)
        if os.path.exists(src_label_path):
            dest_label_path = os.path.join(dest_train_labels, label_file)
            shutil.copy(src_label_path, dest_label_path)

    for file in dev_files:
        src_img_path = os.path.join(source_images_dir, file)
        dest_img_path = os.path.join(dest_dev_images, file)
        shutil.copy(src_img_path, dest_img_path)

        label_file = file.rsplit('.', 1)[0] + '.txt'
        src_label_path = os.path.join(source_labels_dir, label_file)
        if os.path.exists(src_label_path):
            dest_label_path = os.path.join(dest_dev_labels, label_file)
            shutil.copy(src_label_path, dest_label_path)

    print(f"Деление завершено: {len(train_files)} изображений для train, {len(dev_files)} для dev.")


if __name__ == "__main__":
    base_dir = './data/475_fire_train'
    source_images_dir = os.path.join(base_dir, 'images')
    source_labels_dir = os.path.join(base_dir, 'labels')

    dest_train_images = os.path.join(base_dir, 'train', 'images')
    dest_train_labels = os.path.join(base_dir, 'train', 'labels')
    dest_dev_images = os.path.join(base_dir, 'dev', 'images')
    dest_dev_labels = os.path.join(base_dir, 'dev', 'labels')

    split_train_dev(source_images_dir, source_labels_dir,
                    dest_train_images, dest_train_labels,
                    dest_dev_images, dest_dev_labels,
                    dev_ratio=0.2, seed=42)




Деление завершено: 914 изображений для train, 228 для dev.


# Первый yaml файл

In [ ]:
import os
import yaml
from ultralytics import YOLO


data_dict_new = {
    "train": "/gdrive/MyDrive/работа/fire_and_smoke_detection/data/475_fire_train/train/images",
    "val": "/gdrive/MyDrive/работа/fire_and_smoke_detection/data/475_fire_train/dev/images",
    "test": "/gdrive/MyDrive/работа/fire_and_smoke_detection/data/474_fire_val/images",
    "nc": 2,
    "names": ["smoke", "fire"]
}

yaml_path_correct = "data_correct.yaml"
with open(yaml_path_correct, "w") as f:
    yaml.dump(data_dict_new, f)

print("YAML конфигурация сохранена в", yaml_path_correct)

YAML конфигурация сохранена в data_correct.yaml


# Обучение моделей


In [ ]:
model = YOLO("yolov8s.pt")


train_results = model.train(
    data=yaml_path_correct,
    epochs=50,
    imgsz=640,
    batch=16
)

NameError: name 'yaml_path' is not defined

In [ ]:
model_1 = YOLO("yolov8m.pt")


train_results_1 = model_1.train(
    data=yaml_path_correct,
    epochs=50,
    imgsz=640,
    batch=16
)


In [ ]:
model_2 = YOLO("yolov8l.pt")


train_results_1 = model_2.train(
    data=yaml_path_correct,
    epochs=50,
    imgsz=640,
    batch=16
)

Модель с самыми лучшими показателями.

In [ ]:
model = YOLO("yolov8m.pt")

train_results = model.train(
    data="yaml_path_correct",
    epochs=100,
    imgsz=640,
    batch=32,
    optimizer="sgd",
    lr0=0.01
)

Модель yolov5 не показала хороших результатов, я решила к ней не возвращаться.


In [ ]:
!git clone https://github.com/ultralytics/yolov5.git
!pip install -r yolov5/requirements.txt

In [ ]:
!python /gdrive/MyDrive/работа/fire_and_smoke_detection/yolov5/train.py --img 640 --batch 32 --epochs 100 --data data_correct.yaml --weights yolov5s.pt --optimizer SGD --hyp /gdrive/MyDrive/работа/fire_and_smoke_detection/custom_hyp.yaml --name fire_detect_exp

Traceback (most recent call last):
  File "/gdrive/MyDrive/работа/fire_and_smoke_detection/yolov5/train.py", line 33, in <module>
    import numpy as np
  File "/usr/local/lib/python3.11/dist-packages/numpy/__init__.py", line 181, in <module>
    from . import lib
  File "/usr/local/lib/python3.11/dist-packages/numpy/lib/__init__.py", line 16, in <module>
    from . import npyio
  File "/usr/local/lib/python3.11/dist-packages/numpy/lib/npyio.py", line 1, in <module>
    from ._npyio_impl import (
  File "/usr/local/lib/python3.11/dist-packages/numpy/lib/_npyio_impl.py", line 17, in <module>
    from . import format
  File "/usr/local/lib/python3.11/dist-packages/numpy/lib/format.py", line 170, in <module>
    from numpy.lib._utils_impl import drop_metadata
  File "<frozen importlib._bootstrap>", line 1176, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1147, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 690, in _load_unlocked
  File "<frozen im

Попытка аугментации (чтобы увеличить число изображений с дымом). Метрики оказались хуже, возможно, было бы уместно доработать и  это было дало метрики больше.

In [ ]:
import os
import cv2
import albumentations as A

train_images_dir = "/gdrive/MyDrive/работа/fire_and_smoke_detection/data/475_fire_train/train/images"
train_labels_dir = "/gdrive/MyDrive/работа/fire_and_smoke_detection/data/475_fire_train/train/labels"
augmented_images_dir = "/gdrive/MyDrive/работа/fire_and_smoke_detection/data/475_fire_train/train_augmented/images"
augmented_labels_dir = "/gdrive/MyDrive/работа/fire_and_smoke_detection/data/475_fire_train/train_augmented/labels"



def yolo_to_corners(bbox):

    x_center, y_center, width, height = bbox
    x_min = x_center - width / 2
    y_min = y_center - height / 2
    x_max = x_center + width / 2
    y_max = y_center + height / 2
    return x_min, y_min, x_max, y_max

def corners_to_yolo(corners):

    x_min, y_min, x_max, y_max = corners
    x_center = (x_min + x_max) / 2
    y_center = (y_min + y_max) / 2
    width = x_max - x_min
    height = y_max - y_min
    return [x_center, y_center, width, height]

def adjust_yolo_bbox(bbox):

    x_min, y_min, x_max, y_max = yolo_to_corners(bbox)
    # Обрезаем каждое значение в диапазон [0, 1]
    x_min = max(0.0, x_min)
    y_min = max(0.0, y_min)
    x_max = min(1.0, x_max)
    y_max = min(1.0, y_max)
    return corners_to_yolo((x_min, y_min, x_max, y_max))

def load_labels(label_path):

    bboxes = []
    labels = []
    if not os.path.exists(label_path):
        return bboxes, labels
    with open(label_path, 'r') as f:
        for line in f.readlines():
            parts = line.strip().split()
            if len(parts) != 5:
                continue
            cls = int(parts[0])
            raw_bbox = list(map(float, parts[1:]))
            # Даже если исходно все значения корректны, из-за округления
            # после преобразования могут получаться незначительные отрицательные значения.
            bbox = adjust_yolo_bbox(raw_bbox)
            bboxes.append(bbox)
            labels.append(cls)
    return bboxes, labels

def save_labels(label_path, bboxes, labels):

    with open(label_path, 'w') as f:
        for cls, bbox in zip(labels, bboxes):
            bbox_str = " ".join(map(str, bbox))
            f.write(f"{cls} {bbox_str}\n")

photometric_transform = A.Compose([
    A.RandomBrightnessContrast(p=0.5),
    A.HueSaturationValue(p=0.5),
    A.GaussianBlur(blur_limit=(3, 3), p=0.3)
], bbox_params=A.BboxParams(format='yolo', label_fields=['labels']))

def process_image(image_path, label_path, out_images_dir, out_labels_dir, num_augments=5):

    image = cv2.imread(image_path)
    if image is None:
        print(f"Не удалось загрузить изображение: {image_path}")
        return
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    bboxes, labels = load_labels(label_path)

    bboxes_class0 = []
    labels_class0 = []
    bboxes_other = []
    labels_other = []
    for bbox, label in zip(bboxes, labels):
        if label == 0:
            bboxes_class0.append(bbox)
            labels_class0.append(label)
        else:
            bboxes_other.append(bbox)
            labels_other.append(label)

    base_filename = os.path.splitext(os.path.basename(image_path))[0]

    for i in range(num_augments):
        try:
            augmented_result = photometric_transform(image=image_rgb, bboxes=bboxes_class0, labels=labels_class0)
        except Exception as e:
            print(f"Ошибка при аугментации {base_filename}_aug_{i}: {e}")
            print("Пропускаем аугментацию для этой версии.")
            continue

        aug_image = augmented_result['image']
        aug_bboxes_class0 = augmented_result['bboxes']
        aug_labels_class0 = augmented_result['labels']

        combined_bboxes = aug_bboxes_class0 + bboxes_other
        combined_labels = aug_labels_class0 + labels_other

        aug_image_bgr = cv2.cvtColor(aug_image, cv2.COLOR_RGB2BGR)
        out_image_path = os.path.join(out_images_dir, f"{base_filename}_aug_{i}.jpg")
        out_label_path = os.path.join(out_labels_dir, f"{base_filename}_aug_{i}.txt")
        cv2.imwrite(out_image_path, aug_image_bgr)
        save_labels(out_label_path, combined_bboxes, combined_labels)
        print(f"Обработано и сохранено: {os.path.basename(out_image_path)}")

for filename in os.listdir(train_images_dir):
    if not filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        continue
    image_path = os.path.join(train_images_dir, filename)
    label_filename = os.path.splitext(filename)[0] + ".txt"
    label_path = os.path.join(train_labels_dir, label_filename)
    process_image(image_path, label_path, augmented_images_dir, augmented_labels_dir, num_augments=5)

In [ ]:
data_dict = {
    "train": "/gdrive/MyDrive/работа/fire_and_smoke_detection/data/475_fire_train/train_augmented/images",
    "val": "/gdrive/MyDrive/работа/fire_and_smoke_detection/data/475_fire_train/dev/images",
    "test": "/gdrive/MyDrive/работа/fire_and_smoke_detection/data/474_fire_val/images",
    "nc": 2,
    "names": ["smoke", "fire"]
}


yaml_path_augmented = "data_augmented.yaml"
with open(yaml_path_augmented, "w") as f:
    yaml.dump(data_dict, f)

print("YAML конфигурация сохранена в", yaml_path_augmented)

In [ ]:
model = YOLO("yolov8m.pt")

train_results = model.train(
    data="data_augmented.yaml",
    epochs=100,
    imgsz=640,
    batch=32,
    optimizer="sgd",
    lr0=0.01
)